In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
import math
from scipy.optimize import minimize
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
from scipy.sparse import hstack
from sklearn.model_selection import GridSearchCV
from scipy.optimize import curve_fit
import seaborn as sns
from scipy import stats
from scipy.stats import poisson
from scipy.special import erf
from scipy.stats import gamma
from sklearn.model_selection import RandomizedSearchCV
import time
from scipy.optimize import newton
from scipy.stats import norm
from itertools import product
import nbimporter
import functions
from tqdm import tqdm

#nn modules
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_networkx
from torch.utils.data import Dataset
import networkx as nx
from torch.nn import Linear
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import DenseGCNConv, global_mean_pool
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import to_dense_batch
from torch_geometric.utils import add_remaining_self_loops

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from graph_nets import blocks

from graph_nets import graphs
from graph_nets import modules
from graph_nets import utils_np
from graph_nets import utils_tf

import networkx as nx
import numpy as np
import sonnet as snt
import tensorflow as tf

from torch_geometric.nn import MessagePassing
from torch_geometric.utils import dense_to_sparse
from torch.utils.data import random_split
import torch.optim as optim

from torch_geometric.nn import Sequential, GCNConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import GATv2Conv
from torch.nn.functional import mse_loss
import pickle

In [ ]:
electron_event_info = pd.read_csv('C:\\Users\\istok\\Downloads\\Electrons\\ele_event_cut_better.csv')
electron_layer_info = pd.read_csv('C:\\Users\\istok\\Downloads\\Electrons\\ele_layer_cut_better.csv')

electron_event_info = electron_event_info.astype('float32')

# Convert all columns in pi0_layer_info to float32
electron_layer_info = electron_layer_info.astype('float32')


psb = electron_layer_info.iloc[:, 1:64] #9 eta * 7 phi
emb1 = electron_layer_info.iloc[:, 64:65+50] #17 eta * 3 phi
emb2 = electron_layer_info.iloc[:, 65+50:65+51+62] #9 eta * 7 phi
emb3 = electron_layer_info.iloc[:, 65+51+62:65+51+62+63] #9 eta * 7 phi
hab1 = electron_layer_info.iloc[:,65+51+62+63:] #9 eta * 7 phi

In [ ]:
def sum_and_restructure(df, df_name, num_segments, segment_size):
    #Convert the DataFrame to a NumPy array for manipulation
    data_array = df.to_numpy()
    
    #Reshape the array to 3D with dimensions: [num_rows, segment_size, num_segments]
    #This groups each set of segments together for each row
    reshaped_array = data_array.reshape(-1, num_segments, segment_size)
    
    #Sum over the last axis (num_segments) to combine the segments
    summed_array = reshaped_array.sum(axis=1)
    
    #Convert the summed array back into a DataFrame with appropriate column names
    column_names = [f"{df_name} Et{i+1}" for i in range(segment_size)]
    summed_df = pd.DataFrame(summed_array, columns=column_names)
    
    return summed_df

In [ ]:
#Apply the function to each dataframe
psb_Et = sum_and_restructure(psb, "psb", 7, 9)
emb1_Et = sum_and_restructure(emb1, "emb1", 3, 17)
emb2_Et = sum_and_restructure(emb2, "emb2", 7, 9)
emb3_Et = sum_and_restructure(emb3, "emb3", 7, 9)
hab1_Et = sum_and_restructure(hab1, "hab1", 7, 9)

In [ ]:
psb_eta = electron_event_info["psb eta"]
emb1_eta = electron_event_info["emb1 eta"]
emb2_eta = electron_event_info["emb2 eta"]
emb3_eta = electron_event_info["emb3 eta"]
hab1_eta = electron_event_info["hab1 eta"]
psb_emb1 = pd.DataFrame({"psb - emb1": (psb_eta - emb1_eta)})
emb1_emb2 = pd.DataFrame({"emb1 - emb2": (emb1_eta - emb2_eta)})
emb2_emb3 = pd.DataFrame({"emb2 - emb3": (emb2_eta - emb3_eta)})
emb3_hab1 = pd.DataFrame({"emb3 - hab1": (emb3_eta - hab1_eta)})

In [ ]:
labels = electron_event_info["z"].to_numpy()
target = torch.tensor(labels, dtype=torch.float32)

In [ ]:
def graph_tuple(graph_Etinfo, graph_etainfo, total_Etinfo):
    graph_nx = nx.MultiDiGraph()
    #nodes
    for i in range(53):  # psb=9 l1=17 l2=9 l3=9 hab1=9
        Et = graph_Etinfo[i]
        eta = graph_etainfo[i]
        total_Et = total_Etinfo[i]
        graph_nx.add_node(i, features=np.array([Et, eta, total_Et]))
    
    graph_tuple = utils_np.networkxs_to_graphs_tuple([graph_nx])

    return graph_tuple

In [ ]:
Et_total = []
for index, row in electron_layer_info.iterrows():
    Et = row[1:].to_numpy()  
    Et_total.append(np.sum(Et))

In [ ]:
Et_total_array = np.array(Et_total).reshape(-1, 1)

#Initialize the MinMaxScaler to scale between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))

#Fit and transform the data to scale it
Et_total_normalized = scaler.fit_transform(Et_total_array)

#Flatten the array back to 1D
Et_total_normalized = Et_total_normalized.flatten()

In [ ]:
Et_total_array.max()

In [ ]:
psb_Et_array = psb_Et.to_numpy()
emb1_Et_array = emb1_Et.to_numpy()
emb2_Et_array = emb2_Et.to_numpy()
emb3_Et_array = emb3_Et.to_numpy()
hab1_Et_array = hab1_Et.to_numpy()

In [ ]:
def create_object(psb_Et_array, emb1_Et_array, emb2_Et_array, emb3_Et_array, hab1_Et_array,
                 psb_eta, emb1_eta, emb2_eta, emb3_eta, hab1_eta,
                 Et_total):
    norm_psb = scaler.fit_transform(psb_Et_array.reshape(-1, 1))
    norm_emb1 = scaler.fit_transform(emb1_Et_array.reshape(-1, 1))
    norm_emb2 = scaler.fit_transform(emb2_Et_array.reshape(-1, 1))
    norm_emb3 = scaler.fit_transform(emb3_Et_array.reshape(-1, 1))
    norm_hab1 = scaler.fit_transform(hab1_Et_array.reshape(-1, 1))

    graph_Etinfo = np.concatenate([norm_psb.flatten(), norm_emb1.flatten(),
                                   norm_emb2.flatten(), norm_emb3.flatten(),
                                   norm_hab1.flatten()])

    graph_etainfo = np.concatenate([([psb_eta] * 9),
                                    ([emb1_eta] * 17),
                                    ([emb2_eta] * 9),
                                    ([emb3_eta] * 9),
                                    ([hab1_eta] * 9)])
    
    total_Etinfo = [Et_total] * 53
    
    return graph_tuple(graph_Etinfo, graph_etainfo, total_Etinfo)

In [ ]:
graphlist = []
for i in range(len(psb_Et)):
    graph = create_object(psb_Et_array[i], emb1_Et_array[i], emb2_Et_array[i], emb3_Et_array[i], hab1_Et_array[i],
                 psb_eta[i], emb1_eta[i], emb2_eta[i], emb3_eta[i], hab1_eta[i],
                 Et_total_normalized[i])
    
    graphlist.append(graph)

In [ ]:
data_objects = []
for i in range(len(graphlist)):
    node_features = graphlist[i].nodes
    data = create_data_object(node_features)
    data_objects.append(data)

In [ ]:
class GraphDataset(Dataset):
    def __init__(self, graph, labels, globals_features):
        self.graph = graph
        self.labels = labels
        self.globals = globals_features  #Store the global features

    def __len__(self):
        return len(self.graph)

    def __getitem__(self, idx):
        graph = self.graph[idx]
        label = self.labels[idx]  #Retrieve the label corresponding to the current graph pair
        global_features = self.globals[idx]  #Retrieve the global features corresponding to the current graph pair
        return graph, label, global_features

In [ ]:
global_features = pd.concat([psb_emb1, emb1_emb2, emb2_emb3, emb3_hab1], axis = 1)
global_features_np = global_features.to_numpy()

global_features_tensor = torch.tensor(global_features_np, dtype=torch.float)

In [ ]:
paired_dataset = GraphDataset(data_objects, target, global_features_tensor)
total_size = len(paired_dataset)

# Define the proportions
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(paired_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, drop_last=True)

## GAT model

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_size):
        super(GAT, self).__init__()
        
        self.conv1 = GATv2Conv(in_channels=3, out_channels=6, heads=3, concat=True)
        self.conv2 = GATv2Conv(in_channels=6*3, out_channels=9, heads=3, concat=True)
        self.conv3 = GATv2Conv(in_channels=9*3, out_channels=3, heads=3, concat=False)
        
        self.fc1 = nn.Linear(63, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size * 2)  
        self.fc3 = nn.Linear(hidden_size * 2, hidden_size * 3) 
        self.fc4 = nn.Linear(hidden_size * 3, hidden_size * 2)
        self.fc5 = nn.Linear(hidden_size * 2, hidden_size) 
        self.fc6 = nn.Linear(hidden_size, 1)  


    def forward(self, x, global_features, edge_index, batch, return_attention_weights=False):
        attention_weights_list = []

        x, attention_weights1 = self.conv1(x, edge_index, return_attention_weights=True)
        attention_weights_list.append(attention_weights1)
        x = F.relu(x)

        x, attention_weights2 = self.conv2(x, edge_index, return_attention_weights=True)
        attention_weights_list.append(attention_weights2)
        x = F.relu(x)

        x, attention_weights3 = self.conv3(x, edge_index, return_attention_weights=True)
        attention_weights_list.append(attention_weights3)
        x = F.relu(x)
        
        x_reshaped = x.view(512, 53, 3)
        
        psb_eta, _= torch.max(x_reshaped[:,:9, 1], dim = 1)
        emb1_eta, _ = torch.max(x_reshaped[:, 9:26, 1], dim = 1)
        emb2_eta, _ = torch.max(x_reshaped[:,26:35, 1], dim = 1)
        emb3_eta, _ = torch.max(x_reshaped[:,35:44, 1], dim = 1)
        hab1_eta, _ = torch.max(x_reshaped[:,44:53, 1], dim = 1)
        
        total_Et = torch.mean(x_reshaped[:,:,2], dim=1)
        
        Et = x_reshaped[:, :, 0] 

        ini_features = torch.stack((psb_eta, emb1_eta, 
                                    emb2_eta, emb3_eta,
                                    hab1_eta, total_Et), dim=1)  # Shape [512, 6]
        
        next_features = torch.cat((ini_features, global_features), dim=1) #shape [512, 10]
        

        final_features = torch.cat((next_features, Et), dim=1)  # Shape [512, 63]

        x = F.relu(self.fc1(final_features))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)

        if return_attention_weights:
            return x, attention_weights_list
        else:
            return x

In [ ]:
num_nodes = 53

#Create a fully connected graph
edges = [(i, j) for i in range(num_nodes) for j in range(num_nodes) if i != j]
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()

In [ ]:
model_fixed_eta = GAT(16)
loss_function = nn.MSELoss()
optimizer = torch.optim.AdamW(model_fixed_eta.parameters(), lr=0.0005)

loss_trend = []
val_trend = []

#example training loop
for epoch in range(20):
    model_fixed_eta.train()
    train_loss = 0
    for i, batch in enumerate(train_loader):
        graph_data, labels, global_features = batch  
        x, adj, batch_info = graph_data.x, graph_data.adj, graph_data.batch

        optimizer.zero_grad()
        
        #Forward pass
        predictions = model_fixed_eta(x, global_features, edges, batch_info) 
        predictions = torch.squeeze(predictions)

        #Compute loss
        loss = loss_function(predictions, labels)
        train_loss += loss.item()

        #Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        if i == 0:  
            print(f"Epoch {epoch+1}, Batch 1, First Prediction: {predictions[0].item()}, First Label: {labels[0].item()}")
        
    model_fixed_eta.eval()  
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            graph_data, labels, global_features = batch  
            x, adj, batch_info = graph_data.x, graph_data.adj, graph_data.batch
            predictions = model_fixed_eta(x, global_features, edges, batch_info) 
            predictions = torch.squeeze(predictions)
            loss = loss_function(predictions, labels)
            val_loss += loss.item()
            
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    
    loss_trend.append(avg_train_loss)
    val_trend.append(avg_val_loss)
    
    if epoch % 10 == 0:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model_fixed_eta.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            # Include any other stuff you need to resume training
        }
    torch.save(checkpoint, f'checkpoint_epoch_{epoch}.pth')
    print(f"Checkpoint saved at epoch {epoch}")
    
    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")